<div style="background-color:#000;"><img src="pqn.png"></img></div>

This code analyzes the SPY ticker data from Yahoo Finance to identify and study 3-day losing streaks. It calculates daily returns, identifies losing streaks, and tracks the days since the last streak. The code further computes future returns following these streaks to evaluate market behavior. This is useful for understanding market patterns and predicting future performance based on historical data.

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

Define the ticker symbol and download historical data from Yahoo Finance

In [ ]:
ticker = "SPY"
data = yf.download(ticker)

Calculate daily returns by finding the difference between consecutive closing prices

In [ ]:
data["return"] = data["Close"].diff()

Identify days with negative returns to find losing days in the dataset

In [ ]:
data["down"] = data["return"] < 0

Identify 3-day losing streaks by checking for three consecutive losing days

In [ ]:
data["3_day_losing_streak"] = (
    data["down"] & data["down"].shift(1) & data["down"].shift(2)
)

Initialize a column to track the number of days since the last 3-day losing streak

In [ ]:
data["days_since_last_streak"] = np.nan

Iterate over the data to calculate the days since the last 3-day losing streak

In [ ]:
last_streak_day = -np.inf  # Initialize with a very large negative value

In [ ]:
for i in range(len(data)):
    if data["3_day_losing_streak"].iloc[i]:
        if i - last_streak_day >= 42:  # Check if it's been at least 42 trading days
            data.loc[data.index[i], "days_since_last_streak"] = i - last_streak_day
        last_streak_day = i

Filter the data to show only the occurrences that meet the criteria

In [ ]:
result = data.dropna(subset=["days_since_last_streak"]).copy()

Calculate future returns following the identified streaks

In [ ]:
result["next_1_day_return"] = data["Close"].shift(-1) / data["Close"] - 1
result["next_5_day_return"] = data["Close"].shift(-5) / data["Close"] - 1
result["next_10_day_return"] = data["Close"].shift(-10) / data["Close"] - 1
result["next_21_day_return"] = data["Close"].shift(-21) / data["Close"] - 1

Print the mean future returns for different time horizons

In [ ]:
cols = [
    "next_1_day_return",
    "next_5_day_return",
    "next_10_day_return",
    "next_21_day_return"
]
print(result[cols].mean())

Plot the proportion of positive returns for the different time horizons

In [ ]:
result[cols].gt(0).mean().plot.bar()

Display the proportion of positive returns for the different time horizons

In [ ]:
result[cols].gt(0).mean()

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.